In [ ]:
!pip install tensorflow keras

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
df = pd.read_csv('/content/combined_featured_dataset.csv')
df.head()

In [ ]:
texts = df['Clean_Message'].astype(str).tolist()
labels = df['label'].values

In [ ]:
MAX_NUM_WORDS = 20000
MAX_SEQUENCE_LENGTH = 500
EMBEDDING_DIM = 100

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, lower=True, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print("Found %s unique tokens." % len(word_index))

In [ ]:
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
y = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)

In [ ]:
model = Sequential([
    Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.1,
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

In [ ]:
print("Classification Report (CNN):\n")
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Benign', 'Phish'], yticklabels=['Benign', 'Phish'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (CNN)")
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training and Validation Accuracy (CNN)')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training and Validation Loss (CNN)')
plt.legend()
plt.show()

In [ ]:
from tensorflow.keras.layers import LSTM

model_cnn_lstm = tf.keras.Sequential([
    Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    tf.keras.layers.Reshape((1, 128)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model_cnn_lstm.compile(loss='binary_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])

model_cnn_lstm.summary()

In [ ]:
history_cnn_lstm = model_cnn_lstm.fit(
    X_train, y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.1,
    verbose=1
)

In [ ]:
y_pred_prob_cnn_lstm = model_cnn_lstm.predict(X_test)
y_pred_cnn_lstm = (y_pred_prob_cnn_lstm >= 0.5).astype(int)

In [ ]:
print("Classification Report (CNN + LSTM):\n")
print(classification_report(y_test, y_pred_cnn_lstm, digits=4))

In [ ]:
cm = confusion_matrix(y_test, y_pred_cnn_lstm)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['Benign', 'Phish'], yticklabels=['Benign', 'Phish'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (CNN + LSTM)")
plt.show()

In [ ]:
plt.plot(history_cnn_lstm.history['accuracy'], label='Train Acc')
plt.plot(history_cnn_lstm.history['val_accuracy'], label='Val Acc')
plt.title('Training and Validation Accuracy (CNN + LSTM)')
plt.legend()
plt.show()

plt.plot(history_cnn_lstm.history['loss'], label='Train Loss')
plt.plot(history_cnn_lstm.history['val_loss'], label='Val Loss')
plt.title('Training and Validation Loss (CNN + LSTM)')
plt.legend()
plt.show()

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
df = pd.read_csv('/content/combined_featured_dataset.csv')
df = df[['Clean_Message', 'label']].dropna()
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

texts = df['Clean_Message'].tolist()
labels = df['label'].tolist()

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, labels, test_size=0.1, stratify=labels, random_state=42
)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)

In [ ]:
class EmailDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

In [ ]:
preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

In [ ]:
print("Classification Report (BERT):\n")
print(classification_report(test_labels, preds, digits=4))

In [ ]:
cm = confusion_matrix(test_labels, preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Benign', 'Phish'], yticklabels=['Benign', 'Phish'])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix (BERT)")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


cnn_acc = accuracy_score(y_test, y_pred)
cnn_prec = precision_score(y_test, y_pred)
cnn_rec = recall_score(y_test, y_pred)
cnn_f1 = f1_score(y_test, y_pred)

cnn_lstm_acc = accuracy_score(y_test, y_pred_cnn_lstm)
cnn_lstm_prec = precision_score(y_test, y_pred_cnn_lstm)
cnn_lstm_rec = recall_score(y_test, y_pred_cnn_lstm)
cnn_lstm_f1 = f1_score(y_test, y_pred_cnn_lstm)

bert_acc = accuracy_score(test_labels, preds)
bert_prec = precision_score(test_labels, preds)
bert_rec = recall_score(test_labels, preds)
bert_f1 = f1_score(test_labels, preds)

In [ ]:
import pandas as pd

data = {
    "Model": ["CNN", "CNN + LSTM", "BERT"],
    "Accuracy": [cnn_acc, cnn_lstm_acc, bert_acc],
    "Precision": [cnn_prec, cnn_lstm_prec, bert_prec],
    "Recall": [cnn_rec, cnn_lstm_rec, bert_rec],
    "F1-score": [cnn_f1, cnn_lstm_f1, bert_f1],
}

df_metrics = pd.DataFrame(data)

In [ ]:
df_metrics = df_metrics.round(4)
df_metrics